In [11]:
import pandas as pd
import json 
import numpy as np
import csv
from pathlib import Path
import ast

# Flatten data

In [12]:
def flatten_all(inputfile, outputfile, 
                fcol_to_flat, scol_to_flat, col_to_del):
    """
    Flatten all the columns with dictionaries in.
    Append all useful column at the end of data frame
    Return a data frame and write into csv
    """
    raw = pd.read_csv(inputfile, delimiter=',', skipinitialspace=True)
    # deal the float column
    f_col = raw[fcol_to_flat].apply(json.loads)
    df = raw.join(pd.DataFrame(f_col.tolist()).astype('float'))
    # deal with the string column
    for col in scol_to_flat:
        s_col = raw[col].apply(json.loads)
        df = df.join(pd.DataFrame(s_col.tolist()))
    
    df = df.drop(col_to_del, axis=1)
    df = df.drop(scol_to_flat, axis=1)
    df = df.drop(fcol_to_flat, axis=1)
    df.to_csv(outputfile, sep=',')
    return df

In [13]:
fcol_to_flat = 'totals'
scol_to_flat = ['device', 'trafficSource', 'geoNetwork']
col_to_del = ['campaignCode', 'cityId', 'latitude', 'longitude', 'networkLocation', 
              'browserSize', 'browserVersion', 'flashVersion', 'language',
              'mobileDeviceBranding', 'mobileDeviceInfo', 
              'mobileDeviceMarketingName', 'mobileDeviceModel',
              'mobileInputSelector','operatingSystemVersion',
              'screenColors', 'screenResolution']
inputfile = './data/train.csv'
outputfile = './data/train_flatten.csv'

In [15]:
df = flatten_all(inputfile, outputfile, 
                 fcol_to_flat, scol_to_flat, col_to_del)
df.columns

Index(['channelGrouping', 'date', 'fullVisitorId', 'sessionId',
       'socialEngagementType', 'visitId', 'visitNumber', 'visitStartTime',
       'bounces', 'hits', 'newVisits', 'pageviews', 'transactionRevenue',
       'visits', 'browser', 'deviceCategory', 'isMobile', 'operatingSystem',
       'adContent', 'adwordsClickInfo', 'campaign', 'isTrueDirect', 'keyword',
       'medium', 'referralPath', 'source', 'city', 'continent', 'country',
       'metro', 'networkDomain', 'region', 'subContinent'],
      dtype='object')

# Transaction Level Feature Engineering

In [17]:
def extract_traffic_source_columns(df):
    df['isTrueDirect']=df['isTrueDirect'].fillna(False)
    df['isTrueDirect']=df['isTrueDirect'].astype(int)
    
    is_source_mall_googleplex = (df['source']=='mall.googleplex.com').astype(int)
    is_source_mall_googleplex.name = 'is_source_googleplex'    
    
    is_source_direct = (df['source']=='(direct)').astype(int)
    is_source_direct.name = 'is_source_direct'
    
    is_source_google = (df['source']=='(direct)').astype(int)
    is_source_google.name = 'is_source_google'
    
    is_medium_referral = (df['medium']=='referral').astype(int)
    is_medium_referral.name = 'is_medium_referral'
    is_medium_none = (df['medium']=='(none)').astype(int)
    is_medium_none.name = 'is_medium_none'
    is_medium_organic = (df['medium']=='organic').astype(int)
    is_medium_organic.name="is_medium_organic"
    
    df = df.drop(columns = ['adContent','adwordsClickInfo','campaign', 'keyword', 'medium', 'referralPath',
                      'source'])
    
    return pd.concat([df,is_source_mall_googleplex,is_source_direct,is_source_google,is_medium_referral,
                    is_medium_none,is_medium_organic],axis=1)


def get_region_columns(df):
    df['is_bayarea'] = (df['metro'] =='San Francisco-Oakland-San Jose CA').astype(int)
    dummies = pd.get_dummies(df['continent'])
    dummies = dummies.drop(['(not set)'],axis=1)
    df = df.drop(['continent','city','subContinent','region','country','metro','networkDomain'],axis=1) 
    new_df = pd.concat([df,dummies],axis=1)
    return new_df


def extract_activities(df):
    df['is_transaction'] = 1 - df['transactionRevenue'].isna()
    df[['bounces', 'hits', 'pageviews', 'visitNumber']] = df[['bounces', 'hits', 'pageviews', 'visitNumber']].fillna(0)
    df['transactionRevenue'] = df['transactionRevenue'].fillna(0)
    return df


def time_value(df):
    import datetime as dt
    time = pd.to_datetime(df.loc[:,'visitStartTime'], unit='s')
    year = time.dt.year
    year = pd.get_dummies(year)
    month = time.dt.month
    q1 = (month <= 3) & (month >= 1)
    q2 = (month <= 6) & (month >= 4)
    q3 = (month <= 9) & (month >= 7)
    q4 = (month <= 12) & (month >= 10)
    dow = time.dt.weekday
    weekday = (dow <= 5) & (dow >=1)
    hour = time.dt.hour
    h_0_6 = (hour <= 6) & (hour >= 0)
    h_7_12 = (hour <= 12) & (hour >= 7)
    h_13_17 = (hour <= 17) & (hour >= 13)
    h_18_23 = (hour <= 23) & (hour >= 18)
    new_df = pd.DataFrame()
    new_df['q1'] = q1.astype(int)
    new_df['q2'] = q2.astype(int)
    new_df['q3'] = q3.astype(int)
    new_df['q4'] = q4.astype(int)
    new_df['weekday'] = weekday.astype(int)
    new_df['h_0_6'] = h_0_6.astype(int)
    new_df['h_7_12'] = h_7_12.astype(int)
    new_df['h_13_17'] = h_13_17.astype(int)
    new_df['h_18_23'] = h_18_23.astype(int)
    return new_df


def device(df):
    browser_chrome = (df['browser']=='Chrome').astype(int)
    is_mobile = df['isMobile'].astype(int)
    system_google = ((df['operatingSystem']=='Android')|(df['operatingSystem']=='Chrome OS')).astype(int)
    new_df = pd.DataFrame()
    new_df['browser_chrome'] = browser_chrome
    new_df['is_mobile'] = is_mobile
    new_df['system_google'] = system_google
    return new_df

In [24]:
def preprocessing(inputfile):
    df = pd.read_csv(inputfile, index_col=0)
    
    df.sort_values(['fullVisitorId', 'visitStartTime'], ascending=True, inplace=True)
    df['last_session_1'] = (
    df['visitStartTime'] - df[['fullVisitorId', 'visitStartTime']].groupby('fullVisitorId')['visitStartTime'].shift(1)
)
    df['next_session_1']=(
    -1*(df['visitStartTime'] - df[['fullVisitorId', 'visitStartTime']].groupby('fullVisitorId')['visitStartTime'].shift(-1))
)

    #df[['next_session_1','last_session_1']] = df[['next_session_1','last_session_1']]/86400
    df[['next_session_1','last_session_1']] = df[['next_session_1','last_session_1']].fillna(0)
    print(df.columns)
    df = extract_traffic_source_columns(df)
    # print("Congrats Brian\n"+df.columns)
    df = get_region_columns(df)
    # print("Congrats Ziyu\n"+df.columns)
    df = extract_activities(df)
    # print("Congrats Yixin\n"+df.columns)
    col_time = time_value(df)
    col_device = device(df)
    
    df_base_model = df[['fullVisitorId', 'is_transaction', 'transactionRevenue', # add three columns
                        'bounces', 'hits', 'pageviews', 'visitNumber',
                       'is_source_googleplex','is_source_direct','is_source_google','is_medium_referral',
                    'is_medium_none','is_medium_organic',
                        'is_bayarea','Africa','Americas','Asia','Europe','Oceania',
                        'next_session_1','last_session_1'
                       ]]
    col_time = col_time.reset_index(drop=True)
    col_device = col_device.reset_index(drop=True)
    df_base_model = df_base_model.reset_index(drop=True)
    df_base_model = pd.concat([df_base_model,col_time],axis=1)
    df_base_model = pd.concat([df_base_model,col_device],axis=1)
    # print("Congrats Xi\n"+df_base_model.columns)
    return df_base_model

In [27]:
inputfile = './data/train_flatten.csv'
df_trans = preprocessing(inputfile)
df_trans.transactionRevenue = df_trans.transactionRevenue.fillna(0)
df_trans.to_csv('./data/train_trans.csv')

Index(['channelGrouping', 'date', 'fullVisitorId', 'sessionId',
       'socialEngagementType', 'visitId', 'visitNumber', 'visitStartTime',
       'bounces', 'hits', 'newVisits', 'pageviews', 'transactionRevenue',
       'visits', 'browser', 'deviceCategory', 'isMobile', 'operatingSystem',
       'adContent', 'adwordsClickInfo', 'campaign', 'isTrueDirect', 'keyword',
       'medium', 'referralPath', 'source', 'city', 'continent', 'country',
       'metro', 'networkDomain', 'region', 'subContinent', 'last_session_1',
       'next_session_1'],
      dtype='object')


In [28]:
df_trans.head()

,fullVisitorId,is_transaction,transactionRevenue,bounces,hits,pageviews,visitNumber,is_source_googleplex,is_source_direct,is_source_google,...,q3,q4,weekday,h_0_6,h_7_12,h_13_17,h_18_23,browser_chrome,is_mobile,system_google
0,4823595352351,0,0.0,1.0,1.0,1.0,1,0,0,0,...,0,1,1,0,0,1,0,1,0,0
1,5103959234087,0,0.0,0.0,10.0,8.0,1,0,0,0,...,1,0,0,0,0,0,1,1,1,1
2,10278554503158,0,0.0,0.0,11.0,8.0,1,0,0,0,...,0,1,1,1,0,0,0,1,0,0
3,20424342248747,0,0.0,0.0,17.0,13.0,1,0,0,0,...,0,1,1,0,1,0,0,1,0,0
4,26722803385797,0,0.0,0.0,3.0,2.0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0


# User Level Feature Engineering

In [29]:
def agg_time(x):
    if len(x)==1:
        return 0
    else:
        return np.log(1+sum(x)/(len(x)-1))

    
def make_agg(v):
    if v in y_c_variables:
        return {
            v: {v + '_' + 'total': lambda x: np.log(1 + sum(x))}
               }
    if v in y_d_variables:
        return {
            v: {v: 'max'}
        }
    if v in c_variables:
        return {
            v: {v + '_' + 'mean': 'mean'}
        }
    if v in i_variables:
        return {
            v: {v + '_' + 'total': 'max'}
        }
    if v in t_variables:
        return {
            v: {v + '_' + 'adjmean': lambda x: agg_time(x)}
        }
    if v in d_variables:
        return {
            v: {
                v + '_' + 'freq': 'sum',
                v + '_' + 'rate': 'mean'
            }
        }

In [30]:
inputfile = './data/train_trans.csv'
df_trans = pd.read_csv(inputfile, index_col=0)
outputfile = './data/train_user.csv'

/anaconda3/envs/ml/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3018: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [32]:
id_variables = ['fullVisitorId']
y_c_variables = ['transactionRevenue']
y_d_variables = ['is_transaction'] # classification
c_variables = ['bounces', 'hits', 'pageviews'] # continuous
i_variables = ['visitNumber']
t_variables = ['next_session_1', 'last_session_1']
d_variables = ['is_source_googleplex',
       'is_source_direct', 'is_source_google', 'is_medium_referral',
       'is_medium_none', 'is_medium_organic', 'is_bayarea', 'Africa',
       'Americas', 'Asia', 'Europe', 'Oceania', 'q1', 'q2', 'q3', 'q4', 'weekday', 'h_0_6', 'h_7_12',
       'h_13_17', 'h_18_23', 'browser_chrome',
       'is_mobile', 'system_google'] # dummies

In [33]:
aggregation = dict()
for v in df_trans.columns:
    if v not in id_variables:
        agg_new = make_agg(v)
        aggregation = {**aggregation, **agg_new}

In [34]:
df_trans = pd.read_csv(inputfile, index_col=0)
df_user = df_trans.groupby('fullVisitorId', as_index=False).agg(aggregation)
df_user.columns = df_user.columns.droplevel(0)
new_col = ['fullVisitorId']
new_col.extend(df_user.columns[1:])
df_user.columns = new_col
df_user.to_csv(outputfile, sep=',')

/anaconda3/envs/ml/lib/python3.6/site-packages/pandas/core/groupby/groupby.py:4656: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


In [37]:
df_user.head()

,fullVisitorId,is_transaction,transactionRevenue_total,bounces_mean,hits_mean,pageviews_mean,visitNumber_total,is_source_googleplex_freq,is_source_googleplex_rate,is_source_direct_freq,...,h_13_17_freq,h_13_17_rate,h_18_23_freq,h_18_23_rate,browser_chrome_freq,browser_chrome_rate,is_mobile_freq,is_mobile_rate,system_google_freq,system_google_rate
0,4823595352351,0,0.0,1.0,1.0,1.0,1,0,0.0,0,...,1,1.0,0,0.0,1,1.0,0,0.0,0,0.0
1,5103959234087,0,0.0,0.0,10.0,8.0,1,0,0.0,0,...,0,0.0,1,1.0,1,1.0,1,1.0,1,1.0
2,10278554503158,0,0.0,0.0,11.0,8.0,1,0,0.0,0,...,0,0.0,0,0.0,1,1.0,0,0.0,0,0.0
3,20424342248747,0,0.0,0.0,17.0,13.0,1,0,0.0,0,...,0,0.0,0,0.0,1,1.0,0,0.0,0,0.0
4,26722803385797,0,0.0,0.0,3.0,2.0,1,0,0.0,0,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
